## 빅 데이터 분석 및 시각화 개론 프로젝트
# 주제: 대중교통 승하차 데이터를 이용한 우이신설선 수요 예측
## 18조: 장민우, 표성준, 한승규
---
주거구역, 환승구역 등 연구 차원에서 지정한 용어는 발표자료를 참고 부탁드립니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

### 우이신설선 환승역(환승구역) 주변의 모든 버스 정류장
- 서울특별시에서 제공하는 지하철역 근처 버스 정류장 리스트를 활용함.
- 우이신설선 정보가 제공되지 않음에 따라 우이신설선은 별도로 검색 (추후 데이터가 직접 제공될 경우 이를 활용 가능)
- 버스정류장 ID 중 0으로 시작하는 ID의 경우 데이터 정제과정 중 (숫자형 자료형으로 변환되어) 자리수가 변동이 있을 수 있으나, 버스정류장 ID는 기본적으로 다섯자리라는 사전 정보를 감안하여 처리함. 
- 환승 수요가 많으므로 기존 버스 이용 시 반드시 이 역에서 환승하지 않았을 가능성이 있으므로, 실제 환승역을 기준으로 앞/뒤 한 역씩 추가적으로 조사하였음.

In [2]:
subwayToBus = pd.read_csv('subwayToBus.csv')
subwayToBus.head()

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
0,1958,310,대화,3,예비군훈련장,36262,177627,464279
1,1958,310,대화,3,대화역,36281,177715,464099
2,1958,310,대화,3,대화역,36601,177718,464073
3,1805,150,송내,1,송내역,46039,178169,443269
4,1805,150,송내,1,송내역,46731,178200,443181


In [3]:
transfer_region_list = list()
subway_list = ['성신여대입구','길음','한성대입구','안암','보문','창신','제기동','신설동','동묘앞']
for station in subway_list:
    transfer_region_list.append(subwayToBus.where(subwayToBus['전철역명'] == station).dropna())
transfer_region_list[0]

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
1088,418.0,418,성신여대입구,4,삼선동주민센터,8176.0,201300.0,454575.0
1089,418.0,418,성신여대입구,4,성신여대입구,8269.0,201435.0,454875.0
1090,418.0,418,성신여대입구,4,성신여대입구,8270.0,201449.0,454884.0
1091,418.0,418,성신여대입구,4,돈암시장입구,8305.0,201461.0,454719.0
1092,418.0,418,성신여대입구,4,돈암시장입구,8306.0,201478.0,454709.0
1093,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8008.0,201576.0,454865.0
1094,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8007.0,201602.0,454913.0


In [4]:
transfer_region = list()
for i in transfer_region_list:
    temp_list = i['정류장ID'].tolist()
    for j in temp_list:
        transfer_region.append(int(j))

### 우이신설선 비환승역(주거구역) 주변 반경 350m 안의 모든 버스 정류장
- 관련 공식 데이터가 준비되지 않아 수동으로 조사함.
- 주거구역이므로 반경 350m(도보 5분) 이내로 기준을 정함.

In [5]:
HomeRegionData = pd.read_csv('HomeRegion.csv')
HomeRegionData.head()

,전철역명,정류장명,정류장ID
0,북한산우이,우이동,9500
1,북한산우이,우이동차고지종점,9294
2,북한산우이,우이동차고지기점,9289
3,북한산우이,우이동도선사입구,9102
4,북한산우이,우이동도선사입구,9101


In [6]:
home_region = list()
temp_list = HomeRegionData['정류장ID'].tolist()
for j in temp_list:
    home_region.append(int(j))

### 환승구역과 주거구역을 잇는 모든 버스 노선들을 검색
- 우이신설선 개통 직전인 2017년 8월, 모든 버스의 정류장별 승하차 인원 데이터를 기반으로, 해당 노선이 지나가는 버스 정류소들을 확인 가능
- 지하철이 운행종료한 후 운행하는 N버스(심야버스) 는 제외하였음

In [7]:
busStationBoarding = pd.read_csv('BUS_STATION_BOARDING_MONTH_201708.csv')
busStationBoarding = busStationBoarding.where(busStationBoarding['사용일자'] == 20170801).dropna()
busStationBoarding.head()

,사용일자,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20170801.0,400,400번(염곡동~시청),102000171.0,03265,국립중앙박물관용산가족공원,130.0,108.0,20170804.0
1,20170801.0,400,400번(염곡동~시청),102000188.0,03282,한국폴리텍1대학,180.0,329.0,20170804.0
2,20170801.0,400,400번(염곡동~시청),102000179.0,03273,서빙고동주민센터,143.0,65.0,20170804.0
3,20170801.0,400,400번(염곡동~시청),102000178.0,03272,서빙고동,37.0,160.0,20170804.0
4,20170801.0,400,400번(염곡동~시청),102000180.0,03274,동빙고동,140.0,152.0,20170804.0


In [8]:
def is_passing_by(region, boarding_df):
    result = boarding_df.where(boarding_df['버스정류장ARS번호'] == '0').dropna()
    for ID in region:
        if ID < 10000:
            IDstr = str(ID).zfill(5)
        else:
            IDstr = str(ID)        
        temp = boarding_df.where(boarding_df['버스정류장ARS번호'] == IDstr).dropna()
        result = result.append(temp,ignore_index=True)

    return result

transfer_region_bus = is_passing_by(transfer_region,busStationBoarding)
transfer_region_bus = transfer_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [9]:
home_region_bus = is_passing_by(home_region, busStationBoarding)
home_region_bus = home_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [10]:
common_line = []
for line in transfer_region_bus:
    if line in home_region_bus:
        if not "N" in line:
            common_line.append(line)
        
print(common_line)

['1162', '2115', '152', '1014', '162', '성북22', '성북20', '143', '104', '151', '171', '109', '1113', '1114', '1164', '7211', '110B', '1213', '153', '110A', '121', '1115', '1128', '101', '144']


### 해당 노선들의 순차적 정류장 목록을 조사
- 해당 정보를 제공하는 웹사이트는 있었으나, 이를 위한 API는 존재하지 않았음.
- 정보가 html이 아닌 javascript로 매번 불러오는 방식이어서 자동화가 매우 힘들었음.
- 따라서 웹사이트를 직접 손으로 크롤링함

In [11]:
buslineStops = pd.read_csv('BuslineStops.csv', encoding="euc_kr")
buslineStopsOriginal = buslineStops.copy()

In [12]:
busline_linenum = buslineStops.columns.tolist()
busline_linelist = list()
for linenum in busline_linenum:
    busline_linelist.append(list())
    current_line = buslineStops[linenum].tolist()
    i = 0
    for station in current_line:
        if type(station) == str:
            busline_linelist[-1].append(station.split('(')[-1].split(')')[0])
            buslineStops[linenum][i] = station.split('(')[-1].split(')')[0]
            i = i + 1

In [13]:
buslineStops.head(20)

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,09211,08367,07418,08258,08161,08846,06591,08161,09194,08113,...,08160,12469,07195,09102,08160,09291,09211,10153,09102,09289
1,09213,08355,07338,08259,08163,08847,08465,08163,09310,08115,...,08161,35160,07194,09103,08161,09137,09213,10154,09103,09102
2,09215,08369,07435,08260,08408,08844,08479,08408,09196,08117,...,08163,12010,07233,09105,08163,09139,09215,10103,10200,09103
3,09218,08370,07433,08262,08165,08842,08494,08165,09111,08119,...,08408,12012,07231,09106,08408,09141,09218,10108,10202,09105
4,09223,08368,07296,08408,08167,08448,08507,08167,09120,08121,...,08165,12111,07229,09107,08165,09156,09223,10110,10204,09106
5,09221,08358,07295,08165,08169,08850,08522,08169,09122,08005,...,08167,12429,07227,09108,08167,09158,09307,10162,10206,09107
6,09219,08361,07294,08167,08263,08463,08539,08119,09134,08007,...,08169,12435,07225,09125,08169,09160,09274,10164,10210,09108
7,09166,08364,07430,08169,08265,08477,08554,08121,09156,08009,...,08116,12453,07223,09126,08119,09162,09295,10166,10212,09120
8,09168,08330,07292,08263,08267,08492,08571,08005,09158,01005,...,08114,12431,07221,09128,08121,09164,09276,10168,10214,09122
9,08171,08265,07290,08265,08269,08505,08584,08007,09160,01003,...,08112,12455,07218,09146,08123,09166,09303,10181,10216,09134


### 순차 정류장 목록을 이용하여 각 버스별로 어느 구간이 우이신설선과 중복되는지를 검출
- home: 주거구역 안의 정류장
- transfer: 환승구역의 정류장
- none: 어느쪽에도 해당되지 않음

In [14]:
busline_common_region = list()
busline_common_df = buslineStops.copy()
for i in range(len(busline_linenum)):
    busline_common_region.append(list())
    current_line = busline_linenum[i]
    j = 0
    for station in busline_linelist[i]:
        if int(station) in home_region:
            busline_common_region[-1].append('home')
            busline_common_df[current_line][j] = 'home'
        elif int(station) in transfer_region:
            busline_common_region[-1].append('transfer')
            busline_common_df[current_line][j] = 'transfer'
        else:
            busline_common_region[-1].append('none')
            busline_common_df[current_line][j] = 'none'
        j = j + 1

In [15]:
busline_common_df

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,none,none,none,none,none,none,none,none,none,none,...,none,none,none,home,none,home,none,none,home,home
1,none,none,none,none,home,home,none,home,none,none,...,none,none,none,home,none,home,none,none,home,home
2,none,none,none,home,home,home,none,home,none,home,...,home,none,none,home,home,home,none,none,home,home
3,home,none,none,home,home,home,none,home,home,home,...,home,none,none,home,home,home,home,none,none,home
4,home,none,none,home,home,home,none,home,home,none,...,home,none,none,home,home,home,home,none,none,home
5,home,none,none,home,home,none,none,home,home,none,...,home,none,none,home,home,home,home,none,none,home
6,home,none,none,home,home,none,none,home,home,transfer,...,home,none,none,home,home,home,home,none,none,home
7,home,home,none,home,home,none,none,none,home,none,...,none,none,none,none,home,home,none,none,none,home
8,none,home,none,home,home,home,none,none,home,none,...,none,none,none,none,none,home,none,none,none,home
9,none,home,none,home,transfer,none,none,transfer,home,none,...,none,none,none,none,transfer,home,none,none,none,home


### 각 노선을 이용하여 사람들이 우이신설선 대신 이용하던 경로와 소요 시간 계산
- 우이신설선 환승역의 주변 1역은 전철 역 간격에 +1을 더하였음.

In [28]:
UILine = ['북한산우이','솔밭공원','4.19민주묘지','가오리','화계','삼양','삼양사거리','솔샘','북한산보국문','정릉','성신여대입구','보문','신설동']
NearTransfer = {'한성대입구':11, '길음':11, '창신':12, '안암':12, '동묘앞':13, '제기동':13}

In [29]:
def station_by_busstopID(ID):
    ID = int(ID)
    result_df = subwayToBus.where(subwayToBus['정류장ID'] == ID).dropna()
    if len(result_df.index) == 0:
        result_df = HomeRegionData.where(HomeRegionData['정류장ID'] == ID).dropna()
    if len(result_df.index) == 0:
        return 'none'
    else:
        
        return result_df['전철역명'].values[0]
def cal_subway_station(startst,endst):
    try:
        startpos = UILine.index(startst)
    except:
        startpos = NearTransfer[startst]
    try:
        endpos = UILine.index(endst)
    except:
        endpos = NearTransfer[endst]
    return endpos-startpos

In [30]:
TimeComparison= pd.DataFrame(columns=['버스번호','시작정류소ID','끝정류소ID','시작역','끝역','사이정류장수','사이역수'])
TimeComparison
for i in range(len(busline_linenum)):
    busline_common_region.append(list())
    current_line = busline_linenum[i]
    j = 0
    homestopsID = []
    homestops = []
    transferstopsID = []
    transferstops = []
    beforeblock = 'none'
    for station in busline_linelist[i]:
        if busline_common_df[current_line][j] == 'home':
            if not len(transferstops) == 0:
                break
            if beforeblock == 'none':
                homestopsID.append([buslineStops[current_line][j],j])
                homestops.append(station_by_busstopID(buslineStops[current_line][j]))
            beforeblock = 'home'
        elif busline_common_df[current_line][j] == 'transfer':
            if not beforeblock == 'transfer':
                if station_by_busstopID(buslineStops[current_line][j]) in transferstops:
                    break
                else:
                    transferstopsID.append([buslineStops[current_line][j],j])
                    transferstops.append(station_by_busstopID(buslineStops[current_line][j]))
                    
        elif busline_common_df[current_line][j] == 'none':
            beforeblock = 'none'
        j = j + 1
    for p in range(len(homestops)):
        for q in range(len(transferstops)):
            temp_df = pd.DataFrame({'버스번호':[current_line],
                                    '시작정류소ID':[homestopsID[p][0]],
                                    '끝정류소ID':[transferstopsID[q][0]],
                                    '시작역':[homestops[p]],
                                    '끝역':[transferstops[q]],
                                    '사이정류장수':[transferstopsID[q][1]-homestopsID[p][1]],
                                    '사이역수': [cal_subway_station(homestops[p],transferstops[q])]
                                    })
            TimeComparison = TimeComparison.append(temp_df, ignore_index = True)
TimeComparison = TimeComparison[['버스번호','시작정류소ID','끝정류소ID','시작역','끝역','사이정류장수','사이역수']]
TimeComparison

,버스번호,시작정류소ID,끝정류소ID,시작역,끝역,사이정류장수,사이역수
0,152,09218,08003,솔샘,길음,8.0,4.0
1,152,09218,08007,솔샘,성신여대입구,10.0,3.0
2,152,09218,08179,솔샘,보문,13.0,4.0
3,152,09218,01243,솔샘,신설동,16.0,5.0
4,152,09218,01233,솔샘,동묘앞,105.0,6.0
5,152,09218,06107,솔샘,제기동,107.0,6.0
6,1162,08364,08269,정릉,성신여대입구,4.0,1.0
7,1014,08260,08269,북한산보국문,성신여대입구,9.0,2.0
8,162,08163,08269,북한산보국문,성신여대입구,8.0,2.0
9,162,08163,08010,북한산보국문,한성대입구,66.0,3.0
